In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import h5py
import numpy as np

In [3]:
#data used for training and experiment is stored in hdf5 format
#only tec025 is divided into training set and verification set and others only have 'data_mat'
f = h5py.File('tec025.hdf5','r') 
train_data = np.array(f['train_mat'])
val_data = np.array(f['val_mat'])
#switch the data from signed distance field into unsigned distance field
train_data[train_data<0] = -train_data[train_data<0]
val_data[val_data<0] = -val_data[val_data<0]
#present the shape of traning data
print(train_data[train_data==0].shape)
print(train_data[train_data>0].shape)
print(train_data.shape)
print(val_data[val_data==0].shape)
print(val_data[val_data>0].shape)
print(val_data.shape)

(53394,)
(15216494,)
(466, 32, 32, 32)
(3531,)
(1503797,)
(46, 32, 32, 32)


In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
class SAE(nn.Module):
    def __init__(self):
        super(SAE,self).__init__()
        self.conv1 = nn.Conv3d(in_channels=1,out_channels=8,kernel_size=2,stride=2,padding=0,bias=False)
        self.conv2 = nn.Conv3d(in_channels=8,out_channels=32,kernel_size=2,stride=2,padding=0,bias=False)
        self.conv3 = nn.Conv3d(in_channels=32,out_channels=64,kernel_size=2,stride=2,padding=0,bias=False)
        self.conv4 = nn.ConvTranspose3d(in_channels=64,out_channels=32,kernel_size=2,stride=2,padding=0,bias=True)
        self.conv5 = nn.ConvTranspose3d(in_channels=32,out_channels=8,kernel_size=2,stride=2,padding=0,bias=True)
        self.conv6 = nn.ConvTranspose3d(in_channels=8,out_channels=1,kernel_size=2,stride=2,padding=0,bias=True)
    def forward(self,x):
        x1 = F.relu(self.conv1(x))
        x1 = F.relu(self.conv2(x1))
        x1 = F.relu(self.conv3(x1))
        x2 = F.relu(self.conv4(x1))
        x2 = F.relu(self.conv5(x2))
        x2 = F.relu(self.conv6(x2))
        return x1,x2

In [6]:
torch.cuda.empty_cache()

In [7]:
net=SAE().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(),lr = 0.01)

In [8]:
torch.cuda.empty_cache()

In [9]:
epoches = 80
batch_size = 4
dataloader = torch.utils.data.DataLoader(dataset = train_data,batch_size = batch_size,shuffle = True,drop_last = False)

In [10]:
for epoch in range(epoches):
    for i,data in enumerate(dataloader):
        data = data.reshape(-1,1,32,32,32)
        data = data.to(device)
        output1,output2 = net(data)
        optimizer.zero_grad()
        loss = criterion(output2,data)
        loss.backward()
        optimizer.step()
        if i%50==0 :
            print("epoch:",epoch)
            print("i:",i)
            print("Loss:",loss.data.cpu())

epoch: 0
i: 0
Loss: tensor(90716.7422)
epoch: 0
i: 50
Loss: tensor(21937.7656)
epoch: 0
i: 100
Loss: tensor(12315.2441)
epoch: 1
i: 0
Loss: tensor(12378.7256)
epoch: 1
i: 50
Loss: tensor(17401.5566)
epoch: 1
i: 100
Loss: tensor(11678.4434)
epoch: 2
i: 0
Loss: tensor(18416.9434)
epoch: 2
i: 50
Loss: tensor(11944.2207)
epoch: 2
i: 100
Loss: tensor(12932.0410)
epoch: 3
i: 0
Loss: tensor(12379.9590)
epoch: 3
i: 50
Loss: tensor(17406.2070)
epoch: 3
i: 100
Loss: tensor(16459.5664)
epoch: 4
i: 0
Loss: tensor(13158.7305)
epoch: 4
i: 50
Loss: tensor(8745.0020)
epoch: 4
i: 100
Loss: tensor(377.9664)
epoch: 5
i: 0
Loss: tensor(183.4705)
epoch: 5
i: 50
Loss: tensor(175.6471)
epoch: 5
i: 100
Loss: tensor(152.7775)
epoch: 6
i: 0
Loss: tensor(138.2201)
epoch: 6
i: 50
Loss: tensor(138.3181)
epoch: 6
i: 100
Loss: tensor(102.9841)
epoch: 7
i: 0
Loss: tensor(146.5344)
epoch: 7
i: 50
Loss: tensor(153.8324)
epoch: 7
i: 100
Loss: tensor(139.4715)
epoch: 8
i: 0
Loss: tensor(72.0358)
epoch: 8
i: 50
Loss: tens

In [6]:
torch.save(net.state_dict(), 'UDF-AE.pkl')

<All keys matched successfully>